In [1]:
# hide pakcage non critical warnings

import warnings

warnings.filterwarnings('ignore')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))


%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import matplotlib.pyplot as plt

import scipy.io as sio

import torch
import torch.nn as nn

import torch.nn.functional as F
import torch.nn.init as init

from tensorboard_logger import Logger

import os
import sys
import random
import pickle
import numpy as np
from easydict import EasyDict as edict


import devkit.api as dk

In [3]:
sys.path.append(
    os.path.abspath(
        os.path.join(os.path.dirname('__file__'), "..", "..")
    ))

In [4]:
import torch
import torch.nn as nn

import torch.nn.functional as F
import torch.nn.init as init

from src.models.dcc.custom_data import DCCPT_data

from src.models.dcc.config import cfg, get_data_dir, get_output_dir, AverageMeter, remove_files_in_dir
from src.models.dcc.SDAE import SDAE
from src.models.dcc.extractSDAE import extractSDAE

In [59]:
args = edict()
args.db = 'cic'
args.dims = [768, 500, 200, 50, 10]
args.numlayers = 4
args.maxepoch = 5
args.stepsize = 5
args.filename = 'traindata'
args.niter = 1000
args.batchsize = 256
args.step = 200

In [42]:
net_SAE.base

[Sequential(),
 Sequential(
   (0): Linear(in_features=768, out_features=500, bias=True)
 ),
 Sequential(
   (0): Linear(in_features=768, out_features=500, bias=True)
   (1): Linear(in_features=500, out_features=200, bias=True)
 ),
 Sequential(
   (0): Linear(in_features=768, out_features=500, bias=True)
   (1): Linear(in_features=500, out_features=200, bias=True)
   (2): Linear(in_features=200, out_features=50, bias=True)
 )]

In [86]:
datadir

'/home/egxxlch/dev/capacity_ai/proof-of-concept/data/raw/cic'

In [93]:
pretrained = sio.loadmat(os.path.join(datadir, 'pretrained.mat'))

In [95]:
pretrained['X'].shape

(28302, 768)

In [94]:
pretrained

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Mon Aug 24 12:51:28 2020',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[5.2098133e-02, 4.2625748e-02, 4.2625748e-02, ..., 6.9468573e-02,
         6.6421226e-02, 4.7902707e-02],
        [4.2501949e-02, 5.3127434e-02, 5.3127434e-02, ..., 1.9050211e-03,
         2.3132400e-03, 1.2246565e-03],
        [4.7361940e-02, 4.2625748e-02, 5.6834329e-02, ..., 6.1981194e-02,
         3.9160531e-02, 3.9808609e-02],
        ...,
        [2.8417164e-02, 9.4723878e-03, 9.4723878e-03, ..., 1.9552687e-02,
         1.5953779e-02, 2.6598826e-02],
        [1.4167316e-01, 1.3813134e-01, 1.2396401e-01, ..., 2.7078515e-01,
         2.3567833e-01, 1.9553681e-01],
        [4.2501949e-02, 4.2501949e-02, 1.0625487e-01, ..., 8.1643759e-04,
         9.5251057e-04, 1.3607294e-04]], dtype=float32),
 'Z': array([[ 0.6284058 , -0.482436  , -0.31349948, ..., -2.8897371 ,
         -1.7002928 ,  0.73084915],
        [ 1.7847667 , -2.6461997 , -2.

In [43]:
# Training
def train(trainloader, net, index, optimizer, epoch, use_cuda, logger):
    losses = AverageMeter()

    print('\nIndex: %d \t Epoch: %d' %(index,epoch))

    net.train()
    for batch_idx, inputs in enumerate(trainloader):
        if use_cuda:
            inputs = inputs.cuda()
        optimizer.zero_grad()
        inputs_Var = Variable(inputs)
        outputs = net(inputs_Var, index)

        # record loss
        losses.update(outputs.item(), inputs.size(0))

        outputs.backward()
        optimizer.step()

    # log to TensorBoard
    if logger:
        logger.log_value('train_loss_{}'.format(index), losses.avg, epoch)


# Saving checkpoint
def save_checkpoint(state, index, filename):
    torch.save(state, filename+'/checkpoint_%d.pth.tar' % index)


In [20]:
datadir = get_data_dir(args.db)
outputdir = get_output_dir(args.db)

In [60]:
trainset = DCCPT_data(root=datadir, filename=args.filename, h5=False)
nepoch = int(np.ceil(np.array(args.niter * args.batchsize, dtype=float) / len(trainset)))
stepsize = int(np.ceil(np.array(args.step * args.batchsize, dtype=float) / len(trainset)))


In [61]:
print(len(trainset), nepoch, stepsize)

28302 10 2


In [62]:
maxepoch = [args.maxepoch]*args.numlayers + [args.maxepoch*2]
stepsize = [stepsize]*(args.numlayers+1)

In [63]:
maxepoch

[5, 5, 5, 5, 10]

In [64]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=args.batchsize, shuffle=True)

In [35]:
trainset.train_data.shape

(28302, 768)

In [50]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.autograd import Variable

In [66]:
index

4

In [75]:
net = SDAE(dim=args.dims, dropout=0.2, slope=0.0)
lr = 1
startlayer = 0
use_cuda = False
logger = None
numlayers = args.numlayers
for index in range(startlayer, numlayers+1):
    # Freezing previous layer weights
    if index < numlayers:
        for par in net.base[index].parameters():
            par.requires_grad = False
    else:
        for par in net.base[numlayers-1].parameters():
            par.requires_grad = True
    # setting up optimizer - the bias params should have twice the learning rate w.r.t. weights params
    bias_params = filter(lambda x: ('bias' in x[0]) and (x[1].requires_grad), net.named_parameters())
    bias_params = list(map(lambda x: x[1], bias_params))
    nonbias_params = filter(lambda x: ('bias' not in x[0]) and (x[1].requires_grad), net.named_parameters())
    nonbias_params = list(map(lambda x: x[1], nonbias_params))
    optimizer = optim.SGD([{'params': bias_params, 'lr': 2*lr}, {'params': nonbias_params}],
                          lr=lr, momentum=0.9, weight_decay=0.0, nesterov=True)

    scheduler = lr_scheduler.StepLR(optimizer, step_size=stepsize[index], gamma=0.1)

    print('\nIndex: %d \t Maxepoch: %d'%(index, maxepoch[index]))

    for epoch in range(maxepoch[index]):
        train(trainloader, net, index, optimizer, epoch, use_cuda, logger)
        scheduler.step()
        # Save checkpoint
        save_checkpoint({'epoch': epoch+1, 'state_dict': net.state_dict(), 'optimizer': optimizer.state_dict()},
                        index, filename=outputdir)


Index: 0 	 Maxepoch: 5

Index: 0 	 Epoch: 0

Index: 0 	 Epoch: 1

Index: 0 	 Epoch: 2

Index: 0 	 Epoch: 3

Index: 0 	 Epoch: 4

Index: 1 	 Maxepoch: 5

Index: 1 	 Epoch: 0

Index: 1 	 Epoch: 1

Index: 1 	 Epoch: 2

Index: 1 	 Epoch: 3

Index: 1 	 Epoch: 4

Index: 2 	 Maxepoch: 5

Index: 2 	 Epoch: 0

Index: 2 	 Epoch: 1

Index: 2 	 Epoch: 2

Index: 2 	 Epoch: 3

Index: 2 	 Epoch: 4

Index: 3 	 Maxepoch: 5

Index: 3 	 Epoch: 0

Index: 3 	 Epoch: 1

Index: 3 	 Epoch: 2

Index: 3 	 Epoch: 3

Index: 3 	 Epoch: 4

Index: 4 	 Maxepoch: 10

Index: 4 	 Epoch: 0

Index: 4 	 Epoch: 1

Index: 4 	 Epoch: 2

Index: 4 	 Epoch: 3

Index: 4 	 Epoch: 4

Index: 4 	 Epoch: 5

Index: 4 	 Epoch: 6

Index: 4 	 Epoch: 7

Index: 4 	 Epoch: 8

Index: 4 	 Epoch: 9


In [76]:
net.state_dict()

OrderedDict([('enc_0.weight',
              tensor([[-1.1434e-02, -1.9594e-03,  3.4558e-03,  ..., -4.7059e-03,
                        1.6810e-03, -4.9678e-03],
                      [-3.6852e-03,  3.6995e-03,  1.0262e-03,  ...,  2.2796e-02,
                        8.6157e-03,  6.7976e-03],
                      [-4.7436e-03, -7.0327e-03, -1.5589e-02,  ...,  2.1805e-02,
                        1.4689e-02, -3.9864e-05],
                      ...,
                      [ 1.0821e-02,  1.5491e-02, -1.2256e-02,  ..., -1.8414e-03,
                       -1.7318e-02,  6.5222e-03],
                      [ 2.1018e-02, -1.4286e-04, -2.5896e-03,  ...,  1.9366e-03,
                        1.9253e-02, -4.8882e-03],
                      [-8.6705e-03, -1.9355e-02,  6.1143e-03,  ...,  1.3763e-02,
                        2.7510e-02, -2.2833e-03]])),
             ('enc_0.bias',
              tensor([-2.4845e-03, -9.0984e-04, -4.4595e-02, -5.5412e-04,  3.8172e-03,
                       1.1094e-02,  4.1

In [25]:
net_extract_SAE = extractSDAE(dim=[768, 500, 500, 2000, 10], slope=0.0)

In [29]:
datadir = get_data_dir("cic")
outputdir = get_output_dir("cic")

In [81]:
outputdir

'/home/egxxlch/dev/capacity_ai/proof-of-concept/data/raw/cic/results'

In [82]:
filename = outputdir+'/checkpoint_4.pth.tar'

In [83]:
filename

'/home/egxxlch/dev/capacity_ai/proof-of-concept/data/raw/cic/results/checkpoint_4.pth.tar'

In [84]:
checkpoint = torch.load(filename)

In [85]:
checkpoint['state_dict']

OrderedDict([('enc_0.weight',
              tensor([[ 0.0004,  0.0057, -0.0155,  ...,  0.0049,  0.0231,  0.0227],
                      [ 0.0043, -0.0155,  0.0073,  ...,  0.0105,  0.0149,  0.0112],
                      [ 0.0048, -0.0128,  0.0005,  ...,  0.0128,  0.0045,  0.0074],
                      ...,
                      [-0.0050,  0.0149,  0.0048,  ..., -0.0103, -0.0176, -0.0071],
                      [-0.0170, -0.0007, -0.0054,  ..., -0.0038,  0.0009, -0.0135],
                      [-0.0158, -0.0003,  0.0034,  ..., -0.0220, -0.0005,  0.0091]])),
             ('enc_0.bias',
              tensor([-7.1300e-02, -7.5619e-04,  6.3634e-03, -4.6052e-03,  3.3409e-02,
                      -1.2298e-01, -2.1704e-02, -1.8009e-02,  4.2505e-02,  8.3473e-03,
                       2.4312e-02, -1.5676e-01,  6.9109e-03,  1.0775e-02,  2.2433e-02,
                      -8.2230e-02, -6.4080e-02, -6.3761e-02,  1.6635e-02, -1.4378e-02,
                      -5.0147e-02,  6.5862e-03, -1.0872e-02,

In [35]:
net_extract_SAE.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [15]:
from torchsummary import summary

In [26]:
summary(net_extract_SAE, ((1, 768)))
print(net_extract_SAE)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 500]         384,500
            Linear-2                  [-1, 500]         250,500
            Linear-3                 [-1, 2000]       1,002,000
            Linear-4                   [-1, 10]          20,010
            Linear-5                 [-1, 2000]          22,000
            Linear-6                  [-1, 500]       1,000,500
            Linear-7                  [-1, 500]         250,500
            Linear-8                  [-1, 768]         384,768
Total params: 3,314,778
Trainable params: 3,314,778
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 12.64
Estimated Total Size (MB): 12.70
----------------------------------------------------------------
extractSDAE(
  (enc_0): Linear(in_fe